In [1]:
import pandas as pd
from apify_client import ApifyClient
from datetime import datetime


apify_client = ApifyClient('apify_api_IY3hoxDRTo2d2iF7NoZXPIOtFlDb8J1cSkPQ')

In [2]:
# Start an actor and wait for it to finish
position_df = pd.DataFrame()
job_titles = ["Data Analyst", "Data Engineer"]#, "Data Scientist", "Software Developer", "Product Manager", "Digital Marketer"]
locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver"} 
# locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver",
#              "w+CAIQICIITW9udHJlYWw=": "Montreal", "w+CAIQICIHQ2FsZ2FyeQ==": "Calgary", "w+CAIQICIIRWRtb250b24=": "Edmonton"}  # UULE

for uule in locations:
    print(locations[uule])
    for jt in job_titles:
        print(jt)
        run_input = {
            "csvFriendlyOutput": True,
            "includeUnfilteredResults": False,
            "maxConcurrency": 10,
            "maxPagesPerQuery": 3,
            "queries": f"https://www.google.com/search?ibp=htl;jobs&q={jt}&uule={uule}",
            "saveHtml": False,
            "saveHtmlToKeyValueStore": False,
        }

        actor_call = apify_client.actor(
            'dan.scraper/google-jobs-scraper').call(run_input=run_input)

        dataset_items = apify_client.dataset(
            actor_call['defaultDatasetId']).list_items().items

        d = pd.DataFrame(dataset_items)
        d["query"] = jt
        d["location"] = locations[uule]
        d["run_time"] = str(datetime.now())

        position_df = pd.concat([position_df, d])

    print("="*30)
    break # Only Run For Toronto

Toronto
Data Analyst
Data Engineer


In [3]:
position_df.groupby("location")["query"].value_counts()

location  query        
Toronto   Data Analyst     20
          Data Engineer    10
Name: count, dtype: int64

In [12]:
#_ = position_df.pop("thumbnail") # not useful, with na values
position_df.to_csv("../data/raw_google_1129.csv", index=False)


In [13]:
position_df.head()

,title,companyName,location,via,description,jobHighlights,relatedLinks,extras,metadata,applyLink,query,run_time
0,Data Analyst (DAD/NACRS Coder) - Decision Supp...,Sunnybrook,Toronto,via LinkedIn,Summary Of Duties\n\nThe Data Analyst Coder is...,[{'items': ['Summary Of Duties The Data Analy...,[{'link': 'https://www.google.com/search?sca_e...,"[7 hours ago, Full-time, No degree mentioned]","{'postedAt': '7 hours ago', 'scheduleType': 'F...",{'title': '.nFg2eb{font-weight:500}.Bi6Ddc{fon...,Data Analyst,2024-03-11 23:56:48.991887
1,Marketing Automation Data Analyst,ClickJobs.io,Toronto,via LinkedIn,data analyst in marketing automation and/or sa...,[{'items': ['data analyst in marketing automat...,[{'link': 'https://www.google.com/search?sca_e...,"[12 hours ago, Full-time]","{'postedAt': '12 hours ago', 'scheduleType': '...","{'title': 'Apply on LinkedIn', 'link': 'https:...",Data Analyst,2024-03-11 23:56:48.991887
2,Intermediate Data Analyst,confidential,Toronto,via Apex Systems,Intermediate Data Analyst\nApex Systems is a g...,[{'items': ['Intermediate Data Analyst Apex Sy...,[{'link': 'https://www.google.com/search?sca_e...,"[6 days ago, Contractor]","{'postedAt': '6 days ago', 'scheduleType': 'Co...","{'title': 'Apply on Apex Systems', 'link': 'ht...",Data Analyst,2024-03-11 23:56:48.991887
3,Data Analyst,Jarvis Consulting Group,Toronto,via Indeed,"• Location: Toronto, ON\n• Type: Contract\n• J...","[{'items': ['• Location: Toronto, ON • Type: C...",[{'link': 'https://www.google.com/search?sca_e...,"[3 days ago, Contractor, No degree mentioned]","{'postedAt': '3 days ago', 'scheduleType': 'Co...","{'title': 'Apply on Indeed', 'link': 'https://...",Data Analyst,2024-03-11 23:56:48.991887
4,Data Analyst,Jacobs,Toronto,via Jacobs,Your Impact:\n\nOur People & Places Solutions ...,[{'items': ['Your Impact: Our People & Places...,"[{'link': 'http://www.jacobs.com/', 'text': 'j...","[18 days ago, Full-time, No degree mentioned]","{'postedAt': '18 days ago', 'scheduleType': 'F...","{'title': 'Apply on Jacobs', 'link': 'https://...",Data Analyst,2024-03-11 23:56:48.991887


In [8]:
from opensearchpy import OpenSearch, helpers

In [9]:
host = 'search-swift-hire-dev-jfmldmym4cfbiwdhwmtuqq6ihy.us-west-2.es.amazonaws.com'
port = 443
auth = ('swift', 'Hire123!') # For testing only. Don't store credentials in code.

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [10]:
index_name = "swift_dev"

if not client.indices.exists(index_name):
    client.indices.create(index=index_name)

def doc_generator(df):
    for i, row in df.iterrows():
        doc = {
            "_index": index_name,
            "_source": row.to_dict(),
        }
        yield doc

helpers.bulk(client, doc_generator(position_df))

print("Data Saved to ES")

Data Saved to ES


In [11]:
position_df["query"].value_counts()

query
Data Analyst     20
Data Engineer    10
Name: count, dtype: int64